# Laboratório 3 - Aprendizado de máquina
Filipe Gomes Arante de Souza

Roteiro Aula 3 Lab
1. Carregue o dataset breast cancer disponível no sklearn e obtenha a distribuição de classes deste
dataset.

In [1]:
import pandas as pd
from sklearn.datasets import load_breast_cancer
import numpy as np

def classDistribution (classData):
    tam = len(classData)
    labels = np.unique(classData)
    
    classDistr = []
    for class_value in labels:   
        cvc = len([1 for i in classData if i == class_value])
        classDistr += [(class_value, cvc/tam)]

    return classDistr

breast = load_breast_cancer()
print(classDistribution(breast.target))


[(0, 0.37258347978910367), (1, 0.6274165202108963)]


2. Use o classificador Vizinho Mais Próximo e uma divisão percentual com 30% dos exemplos do
dataset breast cancer usados para teste para obter o valor da acurácia e f1 deste classificador neste
dataset.

In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

breast_X = breast.data # Matriz de dados
breast_y = breast.target # Lista de rótulos

X_train, X_test, y_train, y_test = train_test_split(breast_X, breast_y, test_size=0.3, random_state=0)

oneNN = KNeighborsClassifier(n_neighbors=1)
oneNN.fit(X_train, y_train)

y_pred = oneNN.predict(X_test)

print ('accuracy = ', oneNN.score(X_test, y_test))


accuracy =  0.9181286549707602


3. Use o classificador Vizinho Mais Próximo e uma validação cruzada com 10 folds no
dataset breast cancer para obter a matriz de confusão, a acurácia, o desvio padrão, o intervalo de
confiança a 95%, a f1 macro, a precision micro e o recall micro deste classificador neste dataset.

In [3]:
from sklearn.model_selection import cross_validate
from scipy import stats
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

oneNNCross = KNeighborsClassifier(n_neighbors=1)
scorings = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
scores = cross_validate(oneNNCross, breast_X, breast_y, 
                        scoring=scorings, cv=10)

scores_acc = scores['test_accuracy']
print("Mean Accuracy: %0.2f Standard Deviation: %0.2f" % 
      (scores_acc.mean(),scores_acc.std()))

scores_prec = scores['test_precision_macro']
print("Mean Macro Precision: %0.2f Standard Deviation: %0.2f" % 
      (scores_prec.mean(),scores_prec.std()))

scores_rec = scores['test_recall_macro']
print("Mean Macro Recall: %0.2f Standard Deviation: %0.2f" % 
      (scores_rec.mean(),scores_rec.std()))

scores_f1 = scores['test_f1_macro']
print("Mean Macro F1: %0.2f Standard Deviation: %0.2f" % 
      (scores_f1.mean(),scores_f1.std()))

inf, sup = stats.norm.interval(0.95, loc=scores_acc.mean(), 
                               scale=scores_acc.std()/np.sqrt(len(scores_acc)))

print('confidence interval: ({}, {})'.format(inf, sup))


Mean Accuracy: 0.92 Standard Deviation: 0.03
Mean Macro Precision: 0.92 Standard Deviation: 0.03
Mean Macro Recall: 0.90 Standard Deviation: 0.03
Mean Macro F1: 0.91 Standard Deviation: 0.03
confidence interval: (0.8970480700035256, 0.9343429074400834)


4. Use o classificador K Vizinhos Mais Próximos com o método indicado (use 10 partições) no
dataset breast cancer para realizar a busca pelo melhor valor de K. Apresente a melhor acurácia
obtida, o desvio padrão,o intervalo de confiança a 95% e o valor de K correspondente.

In [4]:
from sklearn.model_selection import GridSearchCV

parameters = {'n_neighbors': list(range(1,6))}

knn = KNeighborsClassifier()

gs = GridSearchCV(
    knn,
    parameters,
    cv = 10,
)

gs_results = gs.fit(breast_X, breast_y)

print("Best Mean Accuracy: %0.2f" % gs.best_score_)
print("Best Parameter Values: ", gs.best_params_)
print("Grid Search Result Infos: ", gs.cv_results_.keys())

for i in range(5):
  parciais = np.array ([gs.cv_results_['split0_test_score'][i], 
                        gs.cv_results_['split1_test_score'][i], 
                        gs.cv_results_['split2_test_score'][i], 
                        gs.cv_results_['split3_test_score'][i], 
                        gs.cv_results_['split4_test_score'][i], 
                        gs.cv_results_['split5_test_score'][i], 
                        gs.cv_results_['split6_test_score'][i], 
                        gs.cv_results_['split7_test_score'][i], 
                        gs.cv_results_['split8_test_score'][i], 
                        gs.cv_results_['split9_test_score'][i]])
  inf, sup = stats.norm.interval(0.95, loc=parciais.mean(), 
                               scale=parciais.std()/np.sqrt(len(parciais)))
  
  print('parcial {}: {}'.format(i + 1, parciais))
  print('mean: {}'.format(parciais.mean()))
  print('confidence interval: ({}, {})'.format(inf, sup))
  print()

Best Mean Accuracy: 0.93
Best Parameter Values:  {'n_neighbors': 5}
Grid Search Result Infos:  dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_n_neighbors', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'split5_test_score', 'split6_test_score', 'split7_test_score', 'split8_test_score', 'split9_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])
parcial 1: [0.92982456 0.84210526 0.92982456 0.92982456 0.9122807  0.89473684
 0.92982456 0.94736842 0.89473684 0.94642857]
mean: 0.9156954887218045
confidence interval: (0.8970480700035256, 0.9343429074400834)

parcial 2: [0.9122807  0.9122807  0.89473684 0.92982456 0.92982456 0.92982456
 0.9122807  0.89473684 0.87719298 0.92857143]
mean: 0.9121553884711779
confidence interval: (0.9013572251287317, 0.9229535518136242)

parcial 3: [0.9122807  0.85964912 0.89473684 0.94736842 0.94736842 0.94736842
 0.96491228 0.94736842 0

5. Use o classificador K Vizinhos Mais Próximos com o método indicado no dataset breast cancer
para realizar a busca pelo melhor valor de K sem possibilidade de superajuste. Apresente a melhor
acurácia obtida, o desvio padrão e o intervalo de confiança a 95%.

In [5]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score

grade = {'n_neighbors': [1, 3, 5]}

gs = GridSearchCV(estimator=knn, param_grid = parameters, 
                  scoring='accuracy', cv = 10)

rkf = RepeatedStratifiedKFold(n_splits=10, n_repeats=3)
scores = cross_val_score(gs, breast_X, breast_y, scoring='accuracy', 
                         cv = rkf)

print (scores)

mean = scores.mean()
std = scores.std()
inf, sup = stats.norm.interval(0.95, loc=mean, 
                               scale=std/np.sqrt(len(scores)))

print("\nMean Accuracy: %0.2f Standard Deviation: %0.2f" % (mean, std))
print ("Accuracy Confidence Interval (95%%): (%0.2f, %0.2f)\n" % 
       (inf, sup)) 

[0.98245614 0.92982456 0.92982456 0.98245614 0.94736842 0.92982456
 0.92982456 0.9122807  0.85964912 0.96428571 0.89473684 0.92982456
 0.89473684 0.92982456 0.89473684 0.92982456 0.96491228 0.92982456
 0.98245614 0.91071429 0.92982456 0.89473684 0.94736842 0.94736842
 0.98245614 0.92982456 0.96491228 0.87719298 0.89473684 0.94642857]

Mean Accuracy: 0.93 Standard Deviation: 0.03
Accuracy Confidence Interval (95%): (0.92, 0.94)



6. Repita o procedimento proposto no exercício 5 somente trocando o classificador para uma
Árvore de Decisão. Compara o desempenho obtido pelo K Vizinhos Mais Próximos com a Árvore
de Decisão utilizando o teste T e o de Wilcoxon.

In [7]:
from sklearn.tree import DecisionTreeClassifier

dT = DecisionTreeClassifier()

dTGrade = {'max_depth': [2, 3], 'min_samples_split': [2, 4]}

dTGS = GridSearchCV(estimator=dT, param_grid = dTGrade, 
                  scoring='accuracy', cv = 10)

dTScores = cross_val_score(dTGS, breast_X, breast_y, scoring='accuracy', 
                           cv = rkf)

print (dTScores)

dTMean = dTScores.mean()
dTStd = dTScores.std()
dTInf, dTSup = stats.norm.interval(0.95, loc=mean, 
                                   scale=std/np.sqrt(len(scores)))

print("\nMean Accuracy: %0.2f Standard Deviation: %0.2f" % (dTMean, 
                                                            dTStd))
print ("Accuracy Confidence Interval (95%%): (%0.2f, %0.2f)\n" % 
       (dTInf, dTSup))


[0.89473684 0.94736842 0.9122807  0.92982456 0.94736842 0.87719298
 0.9122807  0.9122807  0.89473684 0.92857143 0.9122807  0.85964912
 0.92982456 0.87719298 0.8245614  0.9122807  0.94736842 0.92982456
 0.94736842 0.98214286 0.96491228 0.89473684 0.89473684 0.94736842
 0.9122807  0.96491228 0.92982456 0.85964912 0.92982456 0.94642857]

Mean Accuracy: 0.92 Standard Deviation: 0.03
Accuracy Confidence Interval (95%): (0.92, 0.94)



In [8]:
from scipy.stats import ttest_rel, wilcoxon

print('Paired T Test')
s,p = ttest_rel(scores,dTScores)
print("t: %0.2f p-value: %0.2f\n" % (s,p))


print ('Wilcoxon Test')
s,p = wilcoxon (scores,dTScores)
print("w: %0.2f p-value: %0.2f\n" % (s,p))

Paired T Test
t: 1.95 p-value: 0.06

Wilcoxon Test
w: 82.00 p-value: 0.05



/home/filipe/.local/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
